Install Dependencies

In [ ]:
!pip install torch
!pip install transformers
!pip install tqdm

## Imports and Dataset Class to pre process data

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

# Define the dataset class
class ProductDataset(Dataset):
    def __init__(self, product_data):
        self.product_data = product_data
        self.tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

    def __len__(self):
        return len(self.product_data)

    def __getitem__(self, idx):
        product = self.product_data[idx]

        # Convert list of data into sentences
        input_text = f"Order_ID: {product['Order ID']}, Customer Name: {product['Customer Name']}, Customer Email: {product['Customer Email']}, Order Date: {product['Order Date']}, Total Amount: {product['Total Amount']}, Order Status: {product['Order Status']}\n"

        # GPT tokenizer used to tokensize dataset
        input_ids = self.tokenizer.encode(input_text, truncation=True, max_length=512)
        return torch.tensor(input_ids)

# Load the pre-trained GPT-3.5 model
model = GPT2LMHeadModel.from_pretrained("gpt2")


## Initialise variables and Data

In [ ]:
#@title
epochs = 3
batch_size = 4
learning_rate = 1e-5

product_data = [
    {
        "Order ID": 1,
        "Customer Name": "John Smith",
        "Customer Email": "john@example.com",
        "Order Date": "2023-06-15",
        "Total Amount": "$150",
        "Order Status": "Delivered"
    },
    {
        "Order ID": 2,
        "Customer Name": "Sarah Johnson",
        "Customer Email": "sarah@example.com",
        "Order Date": "2023-06-16",
        "Total Amount": "$75",
        "Order Status": "In Progress"
    },
    {
        "Order ID": 3,
        "Customer Name": "Michael Brown",
        "Customer Email": "michael@example.com",
        "Order Date": "2023-06-17",
        "Total Amount": "$200",
        "Order Status": "Delivered"
    },
    {
        "Order ID": 4,
        "Customer Name": "Emma Davis",
        "Customer Email": "emma@example.com",
        "Order Date": "2023-06-18",
        "Total Amount": "$120",
        "Order Status": "Shipped"
    },
    {
        "Order ID": 5,
        "Customer Name": "David Wilson",
        "Customer Email": "david@example.com",
        "Order Date": "2023-06-19",
        "Total Amount": "$90",
        "Order Status": "Delivered"
    },
    {
        "Order ID": 6,
        "Customer Name": "Emily Lee",
        "Customer Email": "emily@example.com",
        "Order Date": "2023-06-20",
        "Total Amount": "$50",
        "Order Status": "Cancelled"
    },
    {
        "Order ID": 7,
        "Customer Name": "Andrew Clark",
        "Customer Email": "andrew@example.com",
        "Order Date": "2023-06-21",
        "Total Amount": "$180",
        "Order Status": "Delivered"
    },
    {
        "Order ID": 8,
        "Customer Name": "Olivia Hall",
        "Customer Email": "olivia@example.com",
        "Order Date": "2023-06-22",
        "Total Amount": "$100",
        "Order Status": "Shipped"
    },
    {
        "Order ID": 9,
        "Customer Name": "James Taylor",
        "Customer Email": "james@example.com",
        "Order Date": "2023-06-23",
        "Total Amount": "$70",
        "Order Status": "In Progress"
    },
    {
        "Order ID": 10,
        "Customer Name": "Sophia Moore",
        "Customer Email": "sophia@example.com",
        "Order Date": "2023-06-24",
        "Total Amount": "$130",
        "Order Status": "Delivered"
    },
    {
        "Order ID": 11,
        "Customer Name": "Daniel Johnson",
        "Customer Email": "daniel@example.com",
        "Order Date": "2023-06-25",
        "Total Amount": "$95",
        "Order Status": "Shipped"
    },
    {
        "Order ID": 12,
        "Customer Name": "Emma Thompson",
        "Customer Email": "emma.thompson@example.com",
        "Order Date": "2023-06-26",
        "Total Amount": "$120",
        "Order Status": "Delivered"
    },
    {
        "Order ID": 13,
        "Customer Name": "William Harris",
        "Customer Email": "william@example.com",
        "Order Date": "2023-06-27",
        "Total Amount": "$200",
        "Order Status": "Delivered"
    },
    {
        "Order ID": 14,
        "Customer Name": "Grace Martin",
        "Customer Email": "grace@example.com",
        "Order Date": "2023-06-28",
        "Total Amount": "$85",
        "Order Status": "Cancelled"
    },
    {
        "Order ID": 15,
        "Customer Name": "Liam Rodriguez",
        "Customer Email": "liam@example.com",
        "Order Date": "2023-06-29",
        "Total Amount": "$150",
        "Order Status": "Delivered"
    },
    {
        "Order ID": 16,
        "Customer Name": "Oliver Wilson",
        "Customer Email": "oliver@example.com",
        "Order Date": "2023-06-30",
        "Total Amount": "$110",
        "Order Status": "Shipped"
    },
    {
        "Order ID": 17,
        "Customer Name": "Ava Jackson",
        "Customer Email": "ava@example.com",
        "Order Date": "2023-07-01",
        "Total Amount": "$75",
        "Order Status": "In Progress"
    },
    {
        "Order ID": 18,
        "Customer Name": "Mia Thompson",
        "Customer Email": "mia@example.com",
        "Order Date": "2023-07-02",
        "Total Amount": "$95",
        "Order Status": "Delivered"
    },
    {
        "Order ID": 19,
        "Customer Name": "Noah Clark",
        "Customer Email": "noah@example.com",
        "Order Date": "2023-07-03",
        "Total Amount": "$180",
        "Order Status": "Delivered"
    },
    {
        "Order ID": 20,
        "Customer Name": "Sophia Allen",
        "Customer Email": "sophia.allen@example.com",
        "Order Date": "2023-07-04",
        "Total Amount": "$120",
        "Order Status": "Shipped"
    },
    {
        "Order ID": 21,
        "Customer Name": "Elijah Turner",
        "Customer Email": "elijah@example.com",
        "Order Date": "2023-07-05",
        "Total Amount": "$90",
        "Order Status": "Delivered"
    },
    {
        "Order ID": 22,
        "Customer Name": "Isabella Davis",
        "Customer Email": "isabella@example.com",
        "Order Date": "2023-07-06",
        "Total Amount": "$55",
        "Order Status": "Cancelled"
    },
    {
        "Order ID": 23,
        "Customer Name": "Benjamin Young",
        "Customer Email": "benjamin@example.com",
        "Order Date": "2023-07-07",
        "Total Amount": "$160",
        "Order Status": "Delivered"
    },
    {
        "Order ID": 24,
        "Customer Name": "Amelia Walker",
        "Customer Email": "amelia@example.com",
        "Order Date": "2023-07-08",
        "Total Amount": "$105",
        "Order Status": "Shipped"
    },
    {
        "Order ID": 26,
        "Customer Name": "Ethan Thompson",
        "Customer Email": "ethan@example.com",
        "Order Date": "2023-07-09",
        "Total Amount": "$95",
        "Order Status": "Delivered"
    },
    {
        "Order ID": 27,
        "Customer Name": "Ava Rodriguez",
        "Customer Email": "ava@example.com",
        "Order Date": "2023-07-10",
        "Total Amount": "$80",
        "Order Status": "Shipped"
    },
    {
        "Order ID": 28,
        "Customer Name": "Liam Davis",
        "Customer Email": "liam@example.com",
        "Order Date": "2023-07-11",
        "Total Amount": "$120",
        "Order Status": "Delivered"
    },
    {
        "Order ID": 29,
        "Customer Name": "Isabella Clark",
        "Customer Email": "isabella@example.com",
        "Order Date": "2023-07-12",
        "Total Amount": "$90",
        "Order Status": "Delivered"
    },
    {
        "Order ID": 30,
        "Customer Name": "James Johnson",
        "Customer Email": "james@example.com",
        "Order Date": "2023-07-13",
        "Total Amount": "$60",
        "Order Status": "Cancelled"
    },
    {
        "Order ID": 31,
        "Customer Name": "Olivia Wilson",
        "Customer Email": "olivia@example.com",
        "Order Date": "2023-07-14",
        "Total Amount": "$180",
        "Order Status": "Delivered"
    },
    {
        "Order ID": 32,
        "Customer Name": "Benjamin Smith",
        "Customer Email": "benjamin@example.com",
        "Order Date": "2023-07-15",
        "Total Amount": "$110",
        "Order Status": "Shipped"
    },
    {
        "Order ID": 33,
        "Customer Name": "Sophia Martin",
        "Customer Email": "sophia@example.com",
        "Order Date": "2023-07-16",
        "Total Amount": "$75",
        "Order Status": "In Progress"
    },
    {
        "Order ID": 34,
        "Customer Name": "Noah Walker",
        "Customer Email": "noah@example.com",
        "Order Date": "2023-07-17",
        "Total Amount": "$95",
        "Order Status": "Delivered"
    },
    {
        "Order ID": 35,
        "Customer Name": "Emma Harris",
        "Customer Email": "emma@example.com",
        "Order Date": "2023-07-18",
        "Total Amount": "$160",
        "Order Status": "Delivered"
    },
    {
        "Order ID": 36,
        "Customer Name": "William Davis",
        "Customer Email": "william@example.com",
        "Order Date": "2023-07-19",
        "Total Amount": "$120",
        "Order Status": "Shipped"
    },
    {
        "Order ID": 37,
        "Customer Name": "Mia Johnson",
        "Customer Email": "mia@example.com",
        "Order Date": "2023-07-20",
        "Total Amount": "$85",
        "Order Status": "Delivered"
    },
    {
        "Order ID": 38,
        "Customer Name": "James Wilson",
        "Customer Email": "james.wilson@example.com",
        "Order Date": "2023-07-21",
        "Total Amount": "$130",
        "Order Status": "Delivered"
    },
    {
        "Order ID": 39,
        "Customer Name": "Oliver Thompson",
        "Customer Email": "oliver.thompson@example.com",
        "Order Date": "2023-07-22",
        "Total Amount": "$70",
        "Order Status": "In Progress"
    },
    {
        "Order ID": 40,
        "Customer Name": "Sophia Jackson",
        "Customer Email": "sophia.jackson@example.com",
        "Order Date": "2023-07-23",
        "Total Amount": "$120",
        "Order Status": "Delivered"
    },
    {
        "Order ID": 41,
        "Customer Name": "Amelia Turner",
        "Customer Email": "amelia.turner@example.com",
        "Order Date": "2023-07-24",
        "Total Amount": "$100",
        "Order Status": "Shipped"
    },
    {
        "Order ID": 42,
        "Customer Name": "Lucas Allen",
        "Customer Email": "lucas@example.com",
        "Order Date": "2023-07-25",
        "Total Amount": "$80",
        "Order Status": "Delivered"
    },
    {
        "Order ID": 43,
        "Customer Name": "Mia Young",
        "Customer Email": "mia.young@example.com",
        "Order Date": "2023-07-26",
        "Total Amount": "$65",
        "Order Status": "Cancelled"
    },
    {
        "Order ID": 44,
        "Customer Name": "Alexander Davis",
        "Customer Email": "alexander@example.com",
        "Order Date": "2023-07-27",
        "Total Amount": "$150",
        "Order Status": "Delivered"
    },
    {
        "Order ID": 45,
        "Customer Name": "Charlotte Martin",
        "Customer Email": "charlotte@example.com",
        "Order Date": "2023-07-28",
        "Total Amount": "$110",
        "Order Status": "Shipped"
    },
    {
        "Order ID": 46,
        "Customer Name": "Henry Clark",
        "Customer Email": "henry@example.com",
        "Order Date": "2023-07-29",
        "Total Amount": "$90",
        "Order Status": "Delivered"
    },
    {
        "Order ID": 47,
        "Customer Name": "Ella Wilson",
        "Customer Email": "ella@example.com",
        "Order Date": "2023-07-30",
        "Total Amount": "$55",
        "Order Status": "Cancelled"
    },
    {
        "Order ID": 48,
        "Customer Name": "Sebastian Johnson",
        "Customer Email": "sebastian@example.com",
        "Order Date": "2023-07-31",
        "Total Amount": "$160",
        "Order Status": "Delivered"
    },
    {
        "Order ID": 49,
        "Customer Name": "Scarlett Thompson",
        "Customer Email": "scarlett@example.com",
        "Order Date": "2023-08-01",
        "Total Amount": "$105",
        "Order Status": "Shipped"
    },
    {
        "Order ID": 50,
        "Customer Name": "Henry Davis",
        "Customer Email": "henry@example.com",
        "Order Date": "2023-08-02",
        "Total Amount": "$120",
        "Order Status": "Delivered"
    },
    {
        "Order ID": 51,
        "Customer Name": "Ella Clark",
        "Customer Email": "ella@example.com",
        "Order Date": "2023-08-03",
        "Total Amount": "$85",
        "Order Status": "Shipped"
    },
    {
        "Order ID": 52,
        "Customer Name": "Sebastian Wilson",
        "Customer Email": "sebastian@example.com",
        "Order Date": "2023-08-04",
        "Total Amount": "$160",
        "Order Status": "Delivered"
    },
    {
        "Order ID": 53,
        "Customer Name": "Scarlett Thompson",
        "Customer Email": "scarlett@example.com",
        "Order Date": "2023-08-05",
        "Total Amount": "$95",
        "Order Status": "Delivered"
    },
    {
        "Order ID": 54,
        "Customer Name": "Jackson Johnson",
        "Customer Email": "jackson@example.com",
        "Order Date": "2023-08-06",
        "Total Amount": "$60",
        "Order Status": "Cancelled"
    },
    {
        "Order ID": 55,
        "Customer Name": "Amelia Wilson",
        "Customer Email": "amelia@example.com",
        "Order Date": "2023-08-07",
        "Total Amount": "$180",
        "Order Status": "Delivered"
    },
    {
        "Order ID": 56,
        "Customer Name": "Lucas Davis",
        "Customer Email": "lucas@example.com",
        "Order Date": "2023-08-08",
        "Total Amount": "$110",
        "Order Status": "Shipped"
    },
    {
        "Order ID": 57,
        "Customer Name": "Mia Clark",
        "Customer Email": "mia@example.com",
        "Order Date": "2023-08-09",
        "Total Amount": "$75",
        "Order Status": "In Progress"
    },
    {
        "Order ID": 58,
        "Customer Name": "Oliver Walker",
        "Customer Email": "oliver@example.com",
        "Order Date": "2023-08-10",
        "Total Amount": "$95",
        "Order Status": "Delivered"
    },
    {
        "Order ID": 59,
        "Customer Name": "Sophia Harris",
        "Customer Email": "sophia@example.com",
        "Order Date": "2023-08-11",
        "Total Amount": "$160",
        "Order Status": "Delivered"
    },
    {
        "Order ID": 60,
        "Customer Name": "Noah Davis",
        "Customer Email": "noah@example.com",
        "Order Date": "2023-08-12",
        "Total Amount": "$120",
        "Order Status": "Shipped"
    },
    {
        "Order ID": 61,
        "Customer Name": "Emma Johnson",
        "Customer Email": "emma@example.com",
        "Order Date": "2023-08-13",
        "Total Amount": "$85",
        "Order Status": "Delivered"
    },
    {
        "Order ID": 62,
        "Customer Name": "William Thompson",
        "Customer Email": "william@example.com",
        "Order Date": "2023-08-14",
        "Total Amount": "$130",
        "Order Status": "Delivered"
    },
    {
        "Order ID": 63,
        "Customer Name": "Ava Jackson",
        "Customer Email": "ava.jackson@example.com",
        "Order Date": "2023-08-15",
        "Total Amount": "$70",
        "Order Status": "In Progress"
    },
    {
        "Order ID": 64,
        "Customer Name": "James Wilson",
        "Customer Email": "james.wilson@example.com",
        "Order Date": "2023-08-16",
        "Total Amount": "$120",
        "Order Status": "Delivered"
    },
    {
        "Order ID": 65,
        "Customer Name": "Olivia Thompson",
        "Customer Email": "olivia.thompson@example.com",
        "Order Date": "2023-08-17",
        "Total Amount": "$100",
        "Order Status": "Shipped"
    },
    {
        "Order ID": 66,
        "Customer Name": "Benjamin Allen",
        "Customer Email": "benjamin@example.com",
        "Order Date": "2023-08-18",
        "Total Amount": "$80",
        "Order Status": "Delivered"
    },
    {
        "Order ID": 67,
        "Customer Name": "Emily Davis",
        "Customer Email": "emily@example.com",
        "Order Date": "2023-08-19",
        "Total Amount": "$65",
        "Order Status": "Cancelled"
    },
    {
        "Order ID": 68,
        "Customer Name": "Lucas Johnson",
        "Customer Email": "lucas.johnson@example.com",
        "Order Date": "2023-08-20",
        "Total Amount": "$150",
        "Order Status": "Delivered"
    },
    {
        "Order ID": 69,
        "Customer Name": "Mia Martin",
        "Customer Email": "mia.martin@example.com",
        "Order Date": "2023-08-21",
        "Total Amount": "$110",
        "Order Status": "Shipped"
    },
    {
        "Order ID": 70,
        "Customer Name": "Elijah Clark",
        "Customer Email": "elijah.clark@example.com",
        "Order Date": "2023-08-22",
        "Total Amount": "$90",
        "Order Status": "Delivered"
    }
]

## Train and save fine-tuned model

In [ ]:
# Prepare the dataset
dataset = ProductDataset(product_data)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=pad_sequence)

# Fine-tuning loop
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

accumulation_steps = 8  # Number of batches to accumulate gradients over
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(epochs):
    model.train()
    total_loss = 0
    accumulated_loss = 0
    for i, batch in enumerate(dataloader):
        batch = batch.to(device)
        outputs = model.forward(input_ids=batch, labels=batch)
        # loss = outputs.loss / accumulation_steps
        loss = outputs.loss / accumulation_steps / batch_size
        accumulated_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        if (i + 1) % accumulation_steps == 0 or (i + 1) == len(dataloader):
            optimizer.step()
            optimizer.zero_grad()
            total_loss += accumulated_loss
            accumulated_loss = 0

    print(f"Epoch {epoch+1}: Average Loss = {total_loss / len(dataloader):.4f}")

# Save the fine-tuned model
model.save_pretrained("fine_tuned_model")

Epoch 1: Average Loss = nan
Epoch 2: Average Loss = nan
Epoch 3: Average Loss = nan


## Load saved model and get responses for prompts

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load the fine-tuned model and tokenizer
model = GPT2LMHeadModel.from_pretrained("fine_tuned_model")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Add a new padding token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Set the pad token ID
model.config.pad_token_id = tokenizer.pad_token_id

# Generate response from prompt
def generate_response(prompt, max_length=100):
    inputs = tokenizer.encode_plus(prompt, add_special_tokens=True, padding="longest", max_length=max_length, truncation=True, return_tensors="pt")
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)

    output = model.generate(input_ids, attention_mask=attention_mask, max_length=max_length, num_return_sequences=1)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

In [ ]:
prompt = "What is Michael Brown's email id?"
response = generate_response(prompt)
print(response)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


What is Michael Brown's email id?

Michael Brown's email id:

Email:

Email:

Email:

Email:

Email:

Email:

Email:

Email:

Email:

Email:

Email:

Email:

Email:

Email:

Email:

Email:

Email:

Email:

Email:

Email:

Email:
